In [1]:
%matplotlib inline

In [2]:
import urllib.request
import tempfile
import os
from collections import defaultdict
import glob
import csv
import io
import sys
import shutil

import skbio
import qiime2
from qiime2.plugins.feature_classifier.methods import extract_reads
from q2_types.feature_data import DNAIterator
import pandas
import seaborn as sns
import matplotlib
import numpy

In [3]:
data_dir = os.path.expanduser('~/Projects/short-read-tax-assignment-bk/data')
mockrobiota_dir = os.path.expanduser('~/Projects/mockrobiota/data')
ref_dir = os.path.expanduser('~/Data')
mock_dir = os.path.join(data_dir, 'mock-community')
expected_dir = os.path.join(data_dir, 'precomputed-results', 'mock-community')

In [4]:
ref_dbs = [('greengenes',
            os.path.join(ref_dir, 'gg_13_8_otus', 'rep_set', '99_otus.fasta'),
            os.path.join(ref_dir, 'gg_13_8_otus', 'taxonomy', '99_otu_taxonomy.txt')),
           ('unite',
            os.path.join(ref_dir, 'unite_7_1', 'developer', 'sh_refs_qiime_ver7_99_20.11.2016_dev.fasta'),
            os.path.join(ref_dir, 'unite_7_1', 'developer', 'sh_taxonomy_qiime_ver7_99_20.11.2016_dev.txt'))]

refs = {}
taxs = {}
for ref_name, db_file, tax_file in ref_dbs:
    seqs = skbio.io.read(db_file, format='fasta', constructor=skbio.DNA)
    refs[ref_name] = list(seqs)
    tax_map = {}
    with open(tax_file) as tax_fh:
        reader = csv.reader(tax_fh, delimiter='\t')
        for row in reader:
            tax_map[row[0]] = row[1]
        taxs[ref_name] = tax_map

In [5]:
def single_character_matches(char, query, subject):
    query = ((query == char) + (query == 'N')).astype(int)
    subject = ((subject == char) + (subject == 'N')).astype(int)
    return numpy.convolve(query[::-1], subject)

    
def gapless_mismatches(query, subject):
    matches = single_character_matches('A', query, subject)
    matches += single_character_matches('C', query, subject)
    matches += single_character_matches('G', query, subject)
    matches += single_character_matches('T', query, subject)
    return len(query) - matches.max()


def compare_observed_and_expected(observed, expected):
    ids = [obs.metadata['id'] for obs in observed]
    rev_observed = [numpy.array(list(str(obs.reverse_complement()))) for obs in observed]
    observed = [numpy.array(list(str(obs))) for obs in observed]
    expected = list(expected)
    keys = [' '.join([exp.metadata['id'], exp.metadata['description']]).strip() for exp in expected]
    expected = [numpy.array(list(str(exp))) for exp in expected]
    comparison = {}
    for _id, obs, rev in zip(ids, observed, rev_observed):
        forward = min([(gapless_mismatches(obs, exp), key) for exp, key in zip(expected, keys)])
        reverse = min([(gapless_mismatches(rev, exp), key) for exp, key in zip(expected, keys)])
        comparison[_id] = min(forward, reverse)
    return comparison

## Establish expected sequences and taxonomies

In [10]:
for com_dir in glob.glob(os.path.join(expected_dir, '*')):
    if not os.path.isdir(com_dir):
        continue
    community = os.path.basename(com_dir)
    this_mock_dir = os.path.join(mock_dir, community)
    
    # mock-3 to -5 use the same sequences as mock-13
    if community in ('mock-3', 'mock-4', 'mock-5'):
        this_mockrobiota_dir = os.path.join(mockrobiota_dir, 'mock-13')
    else:
        this_mockrobiota_dir = os.path.join(mockrobiota_dir, community)
    
    # find the expected taxonomies
    if os.path.exists(os.path.join(this_mockrobiota_dir, 'greengenes')):
        identifier_file_dir = os.path.join(this_mockrobiota_dir, 'greengenes', '13-8', '99-otus')
        ref_seqs = refs['greengenes']
        taxonomy_map = taxs['greengenes']
    elif os.path.exists(os.path.join(this_mockrobiota_dir, 'unite')):
        identifier_file_dir = os.path.join(this_mockrobiota_dir, 'unite', '7-1', '99-otus')
        ref_seqs = refs['unite']
        taxonomy_map = taxs['unite']
    else:
        raise RuntimeError('could not find identifiers for ' + community)
        
    # set the destinations
    es_fp = os.path.join(this_mock_dir, 'expected-sequences.fasta')
    et_fp = os.path.join(this_mock_dir, 'matched-sequence-taxonomies.tsv')
        
    # check to see whether the expected sequences are precompiled ...
    expected_expected = os.path.join(this_mockrobiota_dir, 'source', 'expected-sequences.fasta')
    if os.path.exists(expected_expected):
        print(community)
        expected_sequences = skbio.io.read(expected_expected, format='fasta', constructor=skbio.DNA)
        expected_ids = {' '.join([s.metadata['id'], s.metadata['description']]).strip() for s in expected_sequences}
        shutil.copy(expected_expected, es_fp)
        est_fp = os.path.join(this_mock_dir, 'expected-sequence-taxonomies.tsv')
        with open(est_fp) as est_fh:
            with open(et_fp, 'w') as et_fh:
                reader = csv.reader(est_fh, delimiter='\t')
                writer = csv.writer(et_fh, delimiter='\t')
                writer.writerow(next(reader))
                for tax_id, tax in reader:
                    for exp_id in expected_ids:
                        if community[5:] == '12':
                            substring = ' '.join(exp_id.split(' ')[:2])
                        elif community[5:] in ('3', '4', '5', '13', '14', '15'):
                            _, substring, _ = exp_id.split('.')
                        elif community[5:] == '16':
                            substring = ' '.join(exp_id.split('_')[:2])
                        if substring in tax_id:
                            writer.writerow([exp_id, tax])
            
    # ... and otherwise derive the expected sequences from the identifiers and references
    else:
        identifier_file = os.path.join(identifier_file_dir, 'database-identifiers.tsv')
        with open(identifier_file) as csvfile:
            identifiers = csv.reader(csvfile, delimiter='\t')
            expected_sequences = []
            for row in identifiers:
                for _id in row[1].split():
                    for seq in ref_seqs:
                        if _id == seq.metadata['id']:
                            expected_sequences.append(seq)
                            break
                    else:
                        raise RuntimeError('could not find ' + _id + ' for ' + community)
                        
        # now write them out for safe-keeping
        skbio.io.write((s for s in expected_sequences), 'fasta', es_fp)
        print(community)
        with open(et_fp, 'w') as et_fh:
            writer = csv.writer(et_fh, delimiter='\t')
            writer.writerow(['Taxonomy', 'Standard Taxonomy'])
            for seq in expected_sequences:
                _id = seq.metadata['id']
                writer.writerow([_id, taxonomy_map[_id]])

mock-1
mock-10
mock-12
mock-13
mock-14
mock-15
mock-16
mock-2
mock-3
mock-4
mock-5
mock-6
mock-7
mock-8
mock-9


## Map sequences that match to taxonomies

In [11]:
for com_dir in glob.glob(os.path.join(expected_dir, '*')):
    if not os.path.isdir(com_dir):
        continue
    community = os.path.basename(com_dir)
    this_mock_dir = os.path.join(mock_dir, community)
    
    # load the taxonomy map
    tax_file = os.path.join(this_mock_dir, 'matched-sequence-taxonomies.tsv')
    with open(tax_file) as tf_fh:
        reader = csv.reader(tf_fh, delimiter='\t')
        next(reader)
        tax_map = {seq:tax for seq, tax in reader}
    
    # load the expected sequences
    expected_file = os.path.join(this_mock_dir, 'expected-sequences.fasta')
    expected_sequences = skbio.io.read(expected_file, format='fasta', constructor=skbio.DNA)
                    
    # compare with the observed sequences
    observed_file = os.path.join(this_mock_dir, 'rep_seqs.fna')
    observed = skbio.io.read(observed_file, format='fasta', constructor=skbio.DNA)
    observed = list(observed)
    mismatches = compare_observed_and_expected(observed, expected_sequences)
    if len(mismatches) != len(observed):
        print('warning: ', len(observed)-len(mismatches), 'out of', len(observed),
              'observations not found for', community)
        
    # save the taxonomies of the matched reads
    print(community)
    max_mismatches = 3
    result_file = os.path.join(this_mock_dir, 'trueish-taxonomies.tsv')
    with open(result_file, 'w') as result_fh:
        writer = csv.writer(result_fh, delimiter='\t')
        writer.writerow(['id', 'taxonomy'])
        for rep_id, (mismatch, exp_id) in mismatches.items():
            if mismatch <= max_mismatches:
                if exp_id in tax_map:
                    writer.writerow([rep_id, tax_map[exp_id]])
                else:
                    writer.writerow([rep_id, exp_id + ' not found'])
            else:
                writer.writerow([rep_id, 'other'])
    
    #feature_file = os.path.join(this_mock_dir, 'feature_table.qza')
    #features = qiime2.Artifact.load(feature_file)
    #features = features.view(pandas.DataFrame)
    #best_mismatches = list(mismatches.values())
    #weighted_mismatches = []
    #for obs in mismatches:
    #    try:
    #        count = int(features[obs])
    #    except TypeError:
    #        count = int(sum(features[obs]))
    #    weighted_mismatches.extend([mismatches[obs]]*count)
    #print(community)
    #sns.distplot(best_mismatches, kde=False, bins=50, axlabel='Number of Mismatches')
    #matplotlib.pyplot.show()
    #sns.distplot(weighted_mismatches, kde=False, bins=50, axlabel='Number of Mismatches')
    #matplotlib.pyplot.show()

mock-1
mock-10
mock-12
mock-13
mock-14
mock-15
mock-16
mock-2
mock-3
mock-4
mock-5
mock-6
mock-7
mock-8
mock-9
